In [2]:
import pandas as pd

/home/joao/virtualenvs/esquizo/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/joao/virtualenvs/esquizo/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [3]:
df = pd.read_csv('conversas_mexidas.csv',dtype=str)

In [4]:
print(df.columns)

Index(['Unnamed: 0', 'Conversa', 'Label'], dtype='object')


In [5]:
df.head()

,Unnamed: 0,Conversa,Label
0,0,podemos fazer isso rápido? roxanne korrine e a...,cmq
1,1,"bem, eu pensei que começaríamos com a pronúnci...",cma
2,2,não o hacking e engasgos e cuspir parte. por f...,cms
3,3,"ok, então que tal experimentarmos uma cozinha ...",csq
4,0,você está me convidando para sair. isso é tão ...,cmq


In [6]:
tst = df[df.Label != '0']

In [7]:
utterance = []
allo = []
tmpList = []
for index, row in tst.iterrows():
    allo.append(row['Conversa'])
    if row['Unnamed: 0'] == '0':
        if len(tmpList) != 0:
            utterance.append(tmpList)
            tmpList = []
    
    tmpList.append({'Conversa': row['Conversa'], 'Label': row['Label']})
    #tmpList.append([row['Conversa'], row['Label']])
        

In [61]:
chars = list({l for word in allo for l in word})

In [63]:
token_dict = {token:k for token, k in zip(chars, range(1,len(chars)+1))}

In [64]:
token_dict

{'a': 1,
 'ú': 2,
 'v': 3,
 '3': 4,
 'f': 5,
 'g': 6,
 'u': 7,
 'ç': 8,
 'ã': 9,
 'ü': 10,
 '5': 11,
 '2': 12,
 's': 13,
 'q': 14,
 'í': 15,
 'i': 16,
 'ê': 17,
 '4': 18,
 'z': 19,
 'l': 20,
 'm': 21,
 'é': 22,
 'â': 23,
 'w': 24,
 'p': 25,
 '7': 26,
 ' ': 27,
 '.': 28,
 '-': 29,
 'n': 30,
 ',': 31,
 'e': 32,
 'r': 33,
 'y': 34,
 'õ': 35,
 'b': 36,
 'd': 37,
 'c': 38,
 'ó': 39,
 '0': 40,
 'h': 41,
 '"': 42,
 '…': 43,
 'o': 44,
 'à': 45,
 'j': 46,
 'ô': 47,
 '!': 48,
 'á': 49,
 't': 50,
 'k': 51,
 'x': 52,
 '?': 53}

In [65]:
end_token = '<end>'
start_decoding_token = '<start>'
pad_token = '<pad>'


token_dict[pad_token] = 0
token_dict[end_token] = len(token_dict)
token_dict[start_decoding_token] = len(token_dict)

token_dict

{'a': 1,
 'ú': 2,
 'v': 3,
 '3': 4,
 'f': 5,
 'g': 6,
 'u': 7,
 'ç': 8,
 'ã': 9,
 'ü': 10,
 '5': 11,
 '2': 12,
 's': 13,
 'q': 14,
 'í': 15,
 'i': 16,
 'ê': 17,
 '4': 18,
 'z': 19,
 'l': 20,
 'm': 21,
 'é': 22,
 'â': 23,
 'w': 24,
 'p': 25,
 '7': 26,
 ' ': 27,
 '.': 28,
 '-': 29,
 'n': 30,
 ',': 31,
 'e': 32,
 'r': 33,
 'y': 34,
 'õ': 35,
 'b': 36,
 'd': 37,
 'c': 38,
 'ó': 39,
 '0': 40,
 'h': 41,
 '"': 42,
 '…': 43,
 'o': 44,
 'à': 45,
 'j': 46,
 'ô': 47,
 '!': 48,
 'á': 49,
 't': 50,
 'k': 51,
 'x': 52,
 '?': 53,
 '<pad>': 0,
 '<end>': 54,
 '<start>': 55}

In [66]:
decoder_dict = {token_dict[k]:k for k in token_dict.keys()}

decoder_dict

{1: 'a',
 2: 'ú',
 3: 'v',
 4: '3',
 5: 'f',
 6: 'g',
 7: 'u',
 8: 'ç',
 9: 'ã',
 10: 'ü',
 11: '5',
 12: '2',
 13: 's',
 14: 'q',
 15: 'í',
 16: 'i',
 17: 'ê',
 18: '4',
 19: 'z',
 20: 'l',
 21: 'm',
 22: 'é',
 23: 'â',
 24: 'w',
 25: 'p',
 26: '7',
 27: ' ',
 28: '.',
 29: '-',
 30: 'n',
 31: ',',
 32: 'e',
 33: 'r',
 34: 'y',
 35: 'õ',
 36: 'b',
 37: 'd',
 38: 'c',
 39: 'ó',
 40: '0',
 41: 'h',
 42: '"',
 43: '…',
 44: 'o',
 45: 'à',
 46: 'j',
 47: 'ô',
 48: '!',
 49: 'á',
 50: 't',
 51: 'k',
 52: 'x',
 53: '?',
 0: '<pad>',
 54: '<end>',
 55: '<start>'}

In [15]:
from torchnlp import text_encoders

In [17]:
#CRIA UM ENCODER BASEADO EM TODAS AS PALAVRAS DO CORPUS
encoder = text_encoders.StaticTokenizerEncoder(allo,tokenize = lambda s:s.split())

In [19]:
len(encoder.vocab)

2491

In [33]:
encoded = []
gen = (i for i in allo)
encoded.append(encoder.encode(gen.__next__()))

encoded[0]

tensor([ 5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22,
        23])

In [42]:
encoded = [encoder.encode(linha) for linha in allo]

In [47]:
encoded[0]

tensor([ 5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22,
        23])

In [48]:
labels = []
for index, row in tst.iterrows():
    labels.append(row['Label'])